In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir("/content/drive/MyDrive/courses/21_fall/ML/project")

In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
import tensorflow as tf
import tensorflow.keras as keras
import sys
import h5py
import numpy as np

In [14]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0, 2, 3, 1))

    return x_data, y_data


In [15]:
def data_preprocess(x_data):
    return x_data/255

In [16]:
clean_data_filename = './clean_validation_data.h5'
clean_testdata_filename = './clean_test_data.h5'
poisoned_data_filename = './sunglasses_poisoned_data.h5'
model_filename = './sunglasses_bd_net.h5'

cl_x_valid, cl_y_valid = data_loader(clean_data_filename)
cl_x_valid = data_preprocess(cl_x_valid)
cl_x_test, cl_y_test = data_loader(clean_testdata_filename)
cl_x_test = data_preprocess(cl_x_test)
bd_x_test, bd_y_test = data_loader(poisoned_data_filename)
bd_x_test = data_preprocess(bd_x_test)

net = keras.models.load_model(model_filename)
net.load_weights('./sunglasses_bd_weights.h5')

In [17]:
import pandas as pd

In [18]:
pool3 = net.get_layer('pool_3').output
pool3_net = keras.models.Model(net.input, pool3)
activation_all = pool3_net.predict(cl_x_valid)
print(activation_all.shape)

(11547, 5, 4, 60)


In [19]:
activation = np.mean(activation_all, axis=(0,1,2))
print(activation.shape)

(60,)


In [20]:
cl_label_or = np.argmax(net.predict(cl_x_valid), axis=1)
or_accuracy = np.mean(np.equal(cl_label_or, cl_y_valid))*100
print('Original Classification accuracy:', or_accuracy)

Original Classification accuracy: 97.88689702953148


Prune the b model according to the activation value of pooling layer. I set the channel threshold to 25, which means when the model will prune 25 channels.

In [21]:
#channel_threshold = 27
acc_threshold = 10

seq_sort = np.argsort(activation)

#conv3 = net.get_layer("conv_3")
#weight,bias = conv3.get_weights()

for i in range(60):
    channel = seq_sort[i]

    conv3 = net.get_layer("conv_3")
    weight,bias = conv3.get_weights()
    weight[:, :, :, channel] = 0. 
    bias[channel] = 0.
    conv3.set_weights((weight,bias))
    print("pruned channel:", channel)
    
    cl_label_v = np.argmax(net.predict(cl_x_valid), axis=1)
    valid_accuracy = np.mean(np.equal(cl_label_v, cl_y_valid))*100
    print('Clean Valid Classification accuracy:', valid_accuracy)

    if valid_accuracy < or_accuracy-acc_threshold:
      net.save("sunglasses pruned model.h5")
      break
    
#net.save("sunglasses pruned model.h5")


pruned channel: 32
Clean Valid Classification accuracy: 97.88689702953148
pruned channel: 49
Clean Valid Classification accuracy: 97.88689702953148
pruned channel: 44
Clean Valid Classification accuracy: 97.88689702953148
pruned channel: 34
Clean Valid Classification accuracy: 97.87823677145579
pruned channel: 12
Clean Valid Classification accuracy: 97.88689702953148
pruned channel: 37
Clean Valid Classification accuracy: 97.8609162553044
pruned channel: 31
Clean Valid Classification accuracy: 97.88689702953148
pruned channel: 36
Clean Valid Classification accuracy: 97.8695765133801
pruned channel: 14
Clean Valid Classification accuracy: 97.80895470685026
pruned channel: 0
Clean Valid Classification accuracy: 97.75699315839613
pruned channel: 42
Clean Valid Classification accuracy: 97.62708928726076
pruned channel: 57
Clean Valid Classification accuracy: 97.60976877110939
pruned channel: 2
Clean Valid Classification accuracy: 97.58378799688231
pruned channel: 1
Clean Valid Classificati

The Pruned G net is the one compare the B net and Pruned B net. The clean acc and attack success rate is shown below.

In [22]:
clean_data_filename = './clean_test_data.h5'
poisoned_data_filename = './sunglasses_poisoned_data.h5'
model_filename = './sunglasses_bd_net.h5'
b_prime_model_filename = './sunglasses pruned model.h5'

cl_x_test, cl_y_test = data_loader(clean_data_filename)
cl_x_test = data_preprocess(cl_x_test)
bd_x_test, bd_y_test = data_loader(poisoned_data_filename)
bd_x_test = data_preprocess(bd_x_test)

net = keras.models.load_model(model_filename)
b_prime_net = keras.models.load_model(b_prime_model_filename)

cl_b = np.argmax(net.predict(cl_x_test), axis=1)
cl_b_p = np.argmax(b_prime_net.predict(cl_x_test), axis=1)
cl_g = []
for x1, x2 in zip(cl_b, cl_b_p):
    if x1 == x2:
        cl_g.append(x1)
    else:
        cl_g.append(1283)

cl_g_acc = np.mean(np.equal(cl_g, cl_y_test))*100
print('Clean Classification accuracy for Pruned G net:', cl_g_acc)
    
    
    
bd_b = np.argmax(net.predict(bd_x_test), axis=1)
bd_b_p = np.argmax(b_prime_net.predict(bd_x_test), axis=1)
bd_g = []
for x1, x2 in zip(bd_b, bd_b_p):
    if x1 == x2:
        bd_g.append(x1)
    else:
        bd_g.append(1283)
        
asr = np.mean(np.equal(bd_g, bd_y_test))*100
print('Attack Success Rate for Pruned G net:', asr)

Clean Classification accuracy for Pruned G net: 86.39906469212782
Attack Success Rate for Pruned G net: 98.12938425565082


Fine tune the pruned model b prime.

In [25]:
train_data_filename = './clean_validation_data.h5'
model_filename = './sunglasses pruned model.h5'

x_train, y_train = data_loader(train_data_filename)
x_train = data_preprocess(x_train)

net = keras.models.load_model(model_filename)

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
net.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

net.fit(x_train, y_train, epochs = 20)
net.save("sunglasses fine-pruning model.h5")

Epoch 1/20
361/361 [==============================] - 5s 14ms/step - loss: 1.1906 - accuracy: 0.7774
Epoch 2/20
361/361 [==============================] - 5s 14ms/step - loss: 0.2679 - accuracy: 0.9363
Epoch 3/20
361/361 [==============================] - 5s 14ms/step - loss: 0.2063 - accuracy: 0.9409
Epoch 4/20
361/361 [==============================] - 5s 14ms/step - loss: 0.1615 - accuracy: 0.9547
Epoch 5/20
361/361 [==============================] - 5s 14ms/step - loss: 0.1616 - accuracy: 0.9571
Epoch 6/20
361/361 [==============================] - 5s 14ms/step - loss: 0.1100 - accuracy: 0.9686
Epoch 7/20
361/361 [==============================] - 5s 14ms/step - loss: 0.0984 - accuracy: 0.9729
Epoch 8/20
361/361 [==============================] - 5s 14ms/step - loss: 0.0821 - accuracy: 0.9790
Epoch 9/20
361/361 [==============================] - 5s 14ms/step - loss: 0.1122 - accuracy: 0.9695
Epoch 10/20
361/361 [==============================] - 5s 14ms/step - loss: 0.0899 - accura

In [27]:
clean_data_filename = './clean_test_data.h5'
poisoned_data_filename = './sunglasses_poisoned_data.h5'
model_filename = './sunglasses_bd_net.h5'
b_prime_model_filename = './sunglasses fine-pruning model.h5'

cl_x_test, cl_y_test = data_loader(clean_data_filename)
cl_x_test = data_preprocess(cl_x_test)
bd_x_test, bd_y_test = data_loader(poisoned_data_filename)
bd_x_test = data_preprocess(bd_x_test)

net = keras.models.load_model(model_filename)
b_prime_net = keras.models.load_model(b_prime_model_filename)

cl_b = np.argmax(net.predict(cl_x_test), axis=1)
cl_b_p = np.argmax(b_prime_net.predict(cl_x_test), axis=1)
cl_g = []
for x1, x2 in zip(cl_b, cl_b_p):
    if x1 == x2:
        cl_g.append(x1)
    else:
        cl_g.append(1283)

cl_g_acc = np.mean(np.equal(cl_g, cl_y_test))*100
print('Clean Classification accuracy for fine-pruning G net:', cl_g_acc)
    
    
    
bd_b = np.argmax(net.predict(bd_x_test), axis=1)
bd_b_p = np.argmax(b_prime_net.predict(bd_x_test), axis=1)
bd_g = []
for x1, x2 in zip(bd_b, bd_b_p):
    if x1 == x2:
        bd_g.append(x1)
    else:
        bd_g.append(1283)
        
asr = np.mean(np.equal(bd_g, bd_y_test))*100
print('Attack Success Rate for fine-pruning G net:', asr)

Clean Classification accuracy for fine-pruning G net: 84.294621979735
Attack Success Rate for fine-pruning G net: 0.03897116134060795
